# Adicionando e Ajeitando informações dos GeoJSONs

In [24]:
import json
import pandas as pd
import glob

## Adicionand População aos Municipios e criando um novo GeoJSON

In [3]:
df_populacao_cidade = pd.read_csv(r'C:\Users\samfs\Documents\GITHUB\Mapas_Brasil\Populacao\Municipio_Populacao_2016.csv')
df_populacao_cidade.head(10)

,Unnamed: 0,Cod_UF,Nome_da_UF,Cod_municipio,Nome_do_municipio,Populacao
0,0,12,Acre,1200013,ACRELÂNDIA,12583
1,1,12,Acre,1200054,ASSIS BRASIL,6173
2,2,12,Acre,1200104,BRASILÉIA,21896
3,3,12,Acre,1200138,BUJARI,8606
4,4,12,Acre,1200179,CAPIXABA,8882
5,5,12,Acre,1200203,CRUZEIRO DO SUL,79902
6,6,12,Acre,1200252,EPITACIOLÂNDIA,15452
7,7,12,Acre,1200302,FEIJÓ,32799
8,8,12,Acre,1200328,JORDÃO,6661
9,9,12,Acre,1200336,MÂNCIO LIMA,15457


In [17]:
df_municipio = df_populacao_cidade[df_populacao_cidade.Cod_municipio == int('1100023')]
df_municipio

,Unnamed: 0,Cod_UF,Nome_da_UF,Cod_municipio,Nome_do_municipio,Populacao
3851,1,11,Rondônia,1100023,ARIQUEMES,93004


In [31]:
## Criar um novo geojson
json_complete = []

def prin(arg):
    print(arg)

def query(codigo):
    df_municipio = df_populacao_cidade[df_populacao_cidade.Cod_municipio == codigo]
    sigla_uf = df_municipio.iloc[0][1]
    uf = df_municipio.iloc[0][2]
    populacao = df_municipio.iloc[0][-1]
    
    #print(sigla_uf,uf,populacao)
    return(sigla_uf,uf,populacao)


def create_doc_elastic(feat):
    documents_result = get_id(feat['properties']['id'])
    print(feat['properties']['id'])
    for index,doc in enumerate(documents_result):
        doc = {
            "type": "Feature",
            "properties": {
                "UF": str(sigla_uf),
                "Estado": str(uf),
                "Municipio": feat['properties']['nome_municipio'],
                "Codigo_Municipio": feat['properties']['cod_municipio'],
                "Populacao": str(populacao)
            },
            "geometry":{
                "type": doc['_source']['geometry']['type'],
                "coordinates": doc['_source']['geometry']['coordinates']
            }
        }
    return(doc)

with open('geo-100-update_cidades.json', encoding='utf-8') as json_file:
    data = json.load(json_file)
    
    for feat in data['features']:
        idd = int(feat['properties']['id'])
        sigla_uf,uf,populacao = query(idd)
        
        try:
            if(len(feat['geometry']['coordinates']) == 0 or feat['geometry']['type'] == 'MultiPolygon'):
                doc = create_doc_elastic(feat)
            else:
                doc = {
                    "type": "Feature",
                    "properties": {
                        "UF": str(sigla_uf),
                        "Estado": str(uf),
                        "Municipio": feat['properties']['nome_municipio'],
                        "Codigo_Municipio": feat['properties']['cod_municipio'],
                        "Populacao": str(populacao)
                    },
                    "geometry":{
                        "type": feat['geometry']['type'],
                        "coordinates": feat['geometry']['coordinates']
                    }
                }

            json_complete.append(doc)
        except:
            doc = create_doc_elastic(feat)
            json_complete.append(doc)
        
print("Fim")
##### Salvar arquivo
name = 'Municipios.geojson'
geojson = {
    "type":"FeatureCollection",
    "features": json_complete
}
with open(name, 'w') as json_file:
    json.dump(geojson, json_file, ensure_ascii=True)

1303205
1507805
1506112
1508308
1500909
2103703
2102606
2109056
2106201
2104909
2112407
2104305
2100832
2503209
2605707
2607208
2906907
3304557
3300100
3302601
3302007
3550407
3520400
3555406
4314407
5003207
5101704
5213103
5220702
Fim
